In [1]:
# auto-reload edited modules
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, "/home/mmurakami/crios_backups/an_helper_functions")

# run the script into the curre nt kernel (vars/functions become available)
%run -i "/home/mmurakami/crios_backups/an_helper_functions/prep_grid_aste_90.py"

(450, 90)
RAC2d (40500,)
(100,)


In [2]:
dirroot = "/scratch3/atnguyen/aste_90x150x60/"
dirgrid = dirroot + "GRID_real8/"
dirgridnb = dirroot + "GRID_noblank/"
runstr= "run_c68v_heffmosm3x_layers_lessmem1_viscAHp5em2_it0000_pk0000000001/"
layers_path = dirroot + runstr
extL = "LAYERS"
dirmask = dirroot + "run_template/input_maskTransport/"
dirbudg = layers_path + "diags/BUDG/"
dirdiags = dirbudg
dirstate = layers_path + "diags/STATE/"
dirlayers = layers_path + "diags/LAYERS/"
dirtrsp = layers_path + "diags/TRSP/"

In [3]:
# try to use rdmds
fileprefix = "/scratch3/atnguyen/aste_90x150x60/"
extBasin='run_template/input_maskTransport/'
filename = fileprefix + extBasin + "GATE_transports_v2_mskBasin.bin"
ind = np.fromfile(filename, dtype=np.int32)  # auto-reads .meta for shape/dtype/order
orig_shape = (ind.shape)

ind2d = ind.reshape(ny,nx)

mymsk = np.full((ny,nx),np.nan)
mymsk[ind2d == 57408.0] = 1

# make this smaller
mymsk[:,27:50] = np.nan
mymsk[:160,12:30] = np.nan
mymsk[160:163,15:30] = np.nan

ind = ind.reshape(ny,nx)

In [4]:
landmsk = np.zeros((ny,nx))
landmsk[hf1 == 0] = 1
landmsk[hf1 == 1] = np.nan

In [5]:
# we don't even need to do a mask really we can just define this as an individual cell at the equator or smth
# plt.pcolormesh(get_aste_tracer(ind,nfx,nfy)[0])

In [6]:
# plt.pcolormesh(ind)

In [7]:
# define random equatorial-ish point
myi = 110
myj = 10

### Load the horizontal advective term again

In [8]:
# bbb.boundsT=squeeze(rdmds([dirrun 'layers2TH']))'; bbb.nT=length(bbb.boundsT)-1;
# bbb.boundsS=squeeze(rdmds([dirrun 'layers1SLT']))';bbb.nS=length(bbb.boundsS)-1;
# bbb.binwidthT=(bbb.boundsT(2:end)-bbb.boundsT(1:end-1));
# bbb.binwidthS=(bbb.boundsS(2:end)-bbb.boundsS(1:end-1));

# bbb.binwidthT1=0.5.*(bbb.binwidthT(2:end)+bbb.binwidthT(1:end-1)); %same as diff(bbb.binmidT)
# bbb.binwidthS1=0.5.*(bbb.binwidthS(2:end)+bbb.binwidthS(1:end-1)); %same as diff(bbb.binmidS)
# bbb.binmidT=(bbb.boundsT(2:end)+bbb.boundsT(1:end-1))./2;
# bbb.binmidS=(bbb.boundsS(2:end)+bbb.boundsS(1:end-1))./2;
# nT=bbb.nT;nS=bbb.nS;nTm1=bbb.nT-1;nSm1=bbb.nS-1;

# try to replicate exactly the An way
boundsT = np.squeeze(rdmds(layers_path + "layers2TH")).ravel()
nT = boundsT.size - 1
boundsS = np.squeeze(rdmds(layers_path + "layers1SLT")).ravel()
nS = boundsS.size - 1

binwidthT = boundsT[1:] - boundsT[:-1]
binwidthS = boundsS[1:] - boundsS[:-1]

binwidthT1 = 0.5 * (binwidthT[1:] + binwidthT[:-1])
binwidthS1 = 0.5 * (binwidthS[1:] + binwidthS[:-1])

binmidT = (boundsT[1:] + boundsT[:-1]) /2
binmidS = (boundsS[1:] + boundsS[:-1]) /2

nT = nT
nS = nS
nTm1 = nT-1
nSm1 = nS-1

In [9]:
# we need to also create the theta and salt bins for this area
tsstr = np.array(["0000000002","0000000003"])
t2 = int(tsstr[1]) # for the offline version

In [10]:
# first just load the ADVh and ADVr terms for temperature
file_name = "budg3d_hflux_set2"
meta_budg3d_hflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_hflux_set2["fldList"])
varnames = np.array(["ADVx_TH","ADVy_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
ADVx_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
ADVy_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])

# now 3d zfluxes
file_name = "budg3d_zflux_set1"
meta_budg3d_zflux_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_zflux_set1["fldList"])
varnames = np.array(["ADVr_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
ADVr_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
ADVr_TH = ADVr_TH.reshape(nz,ny,nx)

hf = mygrid['hFacC']

ADV_hconv = calc_UV_conv_mod(nfx, nfy,get_aste_faces(ADVx_TH.reshape(nz, ny, nx), nfx, nfy),get_aste_faces(ADVy_TH.reshape(nz, ny, nx), nfx, nfy))
ADV_hconv = ADV_hconv * hf   # degC·m^3/s at cell centers (matches: ff.DFh = ff.DFh .* hf)
ADVhT = ADV_hconv

trWtopADV = -(ADVr_TH)

ADVrT = np.zeros((nz,ny,nx),dtype=float)
ADVrT[:-1,:,:] = (trWtopADV[:-1] - trWtopADV[1:])  # this is not the way we did it in the original code but this is the way An has done it so we try

In [11]:
# load the temperature and salt for this timestep
# maybe the order of binning is wrong, we're binning at T2 for the effects throughout T1
# I think this is not the correct way to do things -- reread theta and do binning by volume here
t2 = int(tsstr[1])
t1 = int(tsstr[0])
file_name = "state_3d_set1"
meta_state_3d_set1 = parsemeta(dirstate + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_state_3d_set1["fldList"])
varnames = np.array(["THETA","SALT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
THETA2,its,meta = rdmds(os.path.join(dirstate, file_name),t2,returnmeta=True,rec=recs[0])
THETA1,its,meta = rdmds(os.path.join(dirstate, file_name),t1,returnmeta=True,rec=recs[0])  # this is what we want to bin by, NOT the ttend
SALT2,its,meta = rdmds(os.path.join(dirstate, file_name),t2,returnmeta=True,rec=recs[1])
THETA2 = THETA2.reshape((nz,ny,nx))


In [12]:
# we don't really care about it so maybe we can just use 0, myj and myi